# MOVIE RECOMMENDER SYSTEM USING APRIORI ALGORITHM

The goal of this project is to produce rules of the following form: if a person recommends this set of movies, they will also recommend this movie. 

In [1]:
import pandas as pd


In [2]:
data=pd.read_csv("C:\\Users\\User\\Downloads\\ml-25m\\ratings.csv")

In [3]:
data.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


To do this, we first need to determine if a person recommends a movie. We can do this by creating a new feature Favorable, which is True if the person gave a favorable review to a movie:

In [4]:
data["Favorable"] = data["rating"] > 3
#We will sample our dataset to form a training data. This also helps reduce the size of the dataset that will be searched, making the Apriori algorithm run faster. We obtain all reviews from the first 200 users:
ratings = data[data['userId'].isin(range(200))]

Next, we can create a dataset of only the favorable reviews in our sample:

In [5]:
favorable_ratings = ratings[ratings["Favorable"]]

We will be searching the user’s favorable reviews for our itemsets. So, the next thing we need is the movies which each user has given a favorable rating. We can compute this by grouping the dataset by the UserID and iterating over the movies in each group:

In [6]:
favorable_reviews_by_users = dict((k, frozenset(v.values))
 for k, v in favorable_ratings.groupby("userId")["movieId"])

in the preceding code, we stored the values as a frozenset, allowing us to quickly check if a movie has been rated by a user.

Sets are much faster than lists for this type of operation, and we will use them in a later code.

Finally, we can create a DataFrame that tells us how frequently each movie has been given a favorable review:

In [7]:
num_favorable_by_movie = ratings[["movieId", "Favorable"]].groupby("movieId").sum()

We can see the top five movies by running the following code:

In [8]:
num_favorable_by_movie.sort_values(by="Favorable", ascending=False).head()

,Favorable
movieId,
318,87.0
296,79.0
356,78.0
2571,76.0
593,73.0


Implementing the Apriori algorithm
On the first iteration of Apriori, the newly discovered itemsets will have a length of 2, as they will be supersets of the initial itemsets created in the first step. On the second iteration (after applying the fourth step and going back to step 2), the newly discovered itemsets will have a length of 3. This allows us to quickly identify the newly discovered itemsets, as needed in the second step. We can store our discovered frequent itemsets in a dictionary, where the key is the length of the itemsets. This allows us to quickly access the itemsets of a given length, and therefore the most recently discovered frequent itemsets, with the help of the following code:

In [9]:
frequent_itemsets = {}

We also need to define the minimum support needed for an itemset to be considered frequent. This value is chosen based on the dataset but try different values to see how that affects the result. I recommend only changing it by 10 percent at a time though, as the time the algorithm takes to run will be significantly different! Let’s set a minimum support value:

In [10]:
min_support = 50

To implement the first step of the Apriori algorithm, we create an itemset with each movie individually and test if the itemset is frequent. We use frozenset, as they allow us to perform faster set-based operations later on, and they can also be used as keys in our counting dictionary (normal sets cannot).
Let’s look at the following example of frozenset code:

In [11]:
frequent_itemsets[1] = dict((frozenset((movie_id,)), row["Favorable"])
 for movie_id, row in num_favorable_by_movie.iterrows()
 if row["Favorable"] > min_support)

We implement the second and third steps together for efficiency by creating a function that takes the newly discovered frequent itemsets, creates the supersets, and then tests if they are frequent. First, we set up the function to perform these steps:

In [12]:
from collections import defaultdict
def find_frequent_itemsets(favorable_reviews_by_users, k_1_itemsets, min_support):
    counts = defaultdict(int)
    for user, reviews in favorable_reviews_by_users.items():
        for itemset in k_1_itemsets:
            if itemset.issubset(reviews):
                for other_reviewed_movie in reviews - itemset:
                    current_superset = itemset | frozenset((other_reviewed_movie,))
                    counts[current_superset] += 1
    return dict([(itemset, frequency) for itemset, frequency in counts.items() if frequency >= min_support])

Let’s have a look at the core of this function in detail. We iterate through each user, and each of the previously discovered itemsets, and then check if it is a subset of the current set of reviews, which are stored in k_1_itemsets (note that here, k_1 means k-1). If it is, this means that the user has reviewed each movie in the itemset. This is done by the itemset.issubset(reviews) line.

We can then go through each individual movie that the user has reviewed (that is not already in the itemset), create a superset by combining the itemset with the new movie and record that we saw this superset in our counting dictionary. These are the candidate frequent itemsets for this value of k.

We end our function by testing which of the candidate itemsets have enough support to be considered frequent and return only those that have a support more than our min_support value.

This function forms the heart of our Apriori implementation and we now create a loop that iterates over the steps of the larger algorithm, storing the new itemsets as we increase k from 1 to a maximum value. In this loop, k represents the length of the soon-to-be discovered frequent itemsets, allowing us to access the previously most discovered ones by looking in our frequent_itemsets dictionary using the key k – 1. We create the frequent itemsets and store them in our dictionary by their length. Let’s look at the code:

In [13]:
import sys
for k in range(2, 20):
    
    
 # Generate candidates of length k, using the frequent itemsets of length k-1
 # Only store the frequent itemsets
    cur_frequent_itemsets = find_frequent_itemsets(favorable_reviews_by_users,frequent_itemsets[k-1], min_support)
    if len(cur_frequent_itemsets) == 0:
        print("Did not find any frequent itemsets of length {}".format(k))
        sys.stdout.flush()
        break
    else:
        print("I found {} frequent itemsets of length {}".format(len(cur_frequent_itemsets), k))
    frequent_itemsets[k] = cur_frequent_itemsets

I found 125 frequent itemsets of length 2
I found 439 frequent itemsets of length 3
I found 896 frequent itemsets of length 4
I found 1136 frequent itemsets of length 5
I found 917 frequent itemsets of length 6
I found 470 frequent itemsets of length 7
I found 148 frequent itemsets of length 8
I found 26 frequent itemsets of length 9
I found 2 frequent itemsets of length 10
Did not find any frequent itemsets of length 11


Extracting association rules
After the Apriori algorithm has completed, we have a list of frequent itemsets. These aren’t exactly association rules, but they can easily be converted into these rules. For each itemset, we can generate a number of association rules by setting each movie to be the conclusion and the remaining movies as the premise. 

In [14]:
candidate_rules = []
for itemset_length, itemset_counts in frequent_itemsets.items():
    for itemset in itemset_counts.keys():
        for conclusion in itemset:
            premise = itemset - set((conclusion,))
    candidate_rules.append((premise, conclusion))

In these rules, the first partis the list of movies in the premise, while the number after it is the conclusion. In the first case, if a reviewer recommends movie 79, they are also likely to recommend movie 258.

The process of computing confidence starts by creating dictionaries to store how many times we see the premise leading to the conclusion (a correct example of the rule) and how many times it doesn’t (an incorrect example). We then iterate over all reviews and rules, working out whether the premise of the rule applies and, if it does, whether the conclusion is accurate.

In [15]:
correct_counts = defaultdict(int)
incorrect_counts = defaultdict(int)
for user, reviews in favorable_reviews_by_users.items():
    for candidate_rule in candidate_rules:
        premise, conclusion = candidate_rule
        if premise.issubset(reviews):
            if conclusion in reviews:
                correct_counts[candidate_rule] += 1
            else:
                incorrect_counts[candidate_rule] += 1

We then compute the confidence for each rule by dividing the correct count by the total number of times the rule was seen:

In [16]:
rule_confidence = {candidate_rule:
 (correct_counts[candidate_rule] / float(correct_counts[candidate_rule] +
 incorrect_counts[candidate_rule]))
 for candidate_rule in candidate_rules}

Now we can print the top five rules by sorting this confidence dictionary and printing the results:

In [17]:
from operator import itemgetter
sorted_confidence = sorted(rule_confidence.items(), key=itemgetter(1), reverse=True)
for index in range(5):
 print("Rule #{0}".format(index + 1))
 premise, conclusion = sorted_confidence[index][0]
 print("Rule: If a person recommends {0} they will also recommend {1}".format(premise, conclusion))
 print(" - Confidence: {0:.3f}".format(rule_confidence[(premise, conclusion)]))
 print("")

Rule #1
Rule: If a person recommends frozenset({356, 296, 2858, 2571, 1196, 1198, 593, 50}) they will also recommend 318
 - Confidence: 0.889

Rule #2
Rule: If a person recommends frozenset({356, 260, 296, 2858, 2571, 1196, 1198, 593, 50}) they will also recommend 318
 - Confidence: 0.889

Rule #3
Rule: If a person recommends frozenset({356, 296, 2858, 2571, 1196, 593, 50}) they will also recommend 1198
 - Confidence: 0.818

Rule #4
Rule: If a person recommends frozenset({480, 356, 296, 110, 318}) they will also recommend 527
 - Confidence: 0.692

Rule #5
Rule: If a person recommends frozenset({50, 2571, 356}) they will also recommend 1198
 - Confidence: 0.684



The resulting printout shows only the movie IDs, which isn’t very helpful without the names of the movies also. The dataset came with a file called u.items, which stores the movie names and their corresponding MovieID (as well as other information, such as the genre).

In [18]:
data=pd.read_csv("movies.csv")

Let’s also create a helper function for finding the name of a movie by its ID:

In [19]:
def get_movie_name(movie_id):
 title_object = data[data["movieId"] == movie_id]["title"]
 title = title_object.values[0]
 return title

We can now adjust our previous code for printing out the top rules to also include the titles:

In [20]:
for index in range(5):
 print("Rule #{0}".format(index + 1))
 premise, conclusion = sorted_confidence[index][0]
 premise_names = ", ".join(get_movie_name(idx) for idx in premise)
 conclusion_name = get_movie_name(conclusion)
 print("Rule: If a person recommends {0} they will also recommend {1}".format(premise_names, conclusion_name))
 print(" - Confidence: {0:.3f}".format(rule_confidence[(premise, conclusion)]))
 print("")

Rule #1
Rule: If a person recommends Forrest Gump (1994), Pulp Fiction (1994), American Beauty (1999), Matrix, The (1999), Star Wars: Episode V - The Empire Strikes Back (1980), Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981), Silence of the Lambs, The (1991), Usual Suspects, The (1995) they will also recommend Shawshank Redemption, The (1994)
 - Confidence: 0.889

Rule #2
Rule: If a person recommends Forrest Gump (1994), Star Wars: Episode IV - A New Hope (1977), Pulp Fiction (1994), American Beauty (1999), Matrix, The (1999), Star Wars: Episode V - The Empire Strikes Back (1980), Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981), Silence of the Lambs, The (1991), Usual Suspects, The (1995) they will also recommend Shawshank Redemption, The (1994)
 - Confidence: 0.889

Rule #3
Rule: If a person recommends Forrest Gump (1994), Pulp Fiction (1994), American Beauty (1999), Matrix, The (1999), Star Wars: Episode V - The Empire St